# Keras Multi-Output Classification - Bond Data
Learning exercise: Predicting level_2 and level_3 from bond characteristics

In [ ]:
# Setup
!pip install keras keras-hub --upgrade -q

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"

import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

print(f"Keras version: {keras.__version__}")
print(f"Backend: {keras.backend.backend()}")

In [ ]:
# Load data
# TODO: Upload your bond dataset (CSV file)
# df = pd.read_csv('your_bond_data.csv')

# Quick data check
# print(df.shape)
# print(df.head())
# print(df.info())